Demography 88<br>
Fall 2019<br>
Carl Mason (cmason@berkeley.edu)<br>
##   New world pre contact demography,  population presure, population simulation and mortality --Part 2




### Goals
1. What would  Malthus and Boserup think of  Dobyn's and Crosby
1. Exploring life tables with a data science approach.
1. Apreicating the family life implications of different mortality regimes
### Outline

1. Malthus, Boserup theories of population and development.
    1. Carrying capacity and population in 1491
    1. Pristine Myth and Malthus v Boserup
1.  Dobyns, Crosby in the contex of Malthus and Boserup.
    1. After mortality crises: Malthus v Boserup (Malakoff)
        1. Europe's Black Death 1340s
1. Quantifying death
    1. Life Tables (quick review)
    1. Life expectancy ($e_x$)
    1. Historical trends in mortality
1. Mortality, society and family
    1. Using randomness to compute interesting quantities
1. Lab exercise
    1. mortality levels and generational succession
   



In [ ]:
# Run this cell to import the stuff we'll need
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 
#plt.style.use('fivethirtyeight')
import matplotlib
%matplotlib inline


from datascience import Table
from datascience.predicates import are
from datascience.util import *
from IPython.display import HTML, IFrame, display
datasite="https://courses.demog.berkeley.edu/mason88/data/"
quizsite="https://courses.demog.berkeley.edu/mason88/cgi-bin/quiz.py"
imagesite="https://courses.demog.berkeley.edu/mason88/images"  
def cquiz(qno) : 
    import IPython, requests 
    try:
        sid
    except NameError: 
        print("HEY! did you enter your sid way up at the top of this notebook?")
    Linkit='{0}?qno={1}&sid={2}'.format(quizsite,qno,sid)
    #print(Linkit)
    html = requests.get(Linkit)
    display(IFrame(Linkit, 1000, 300))
    
######################
# Here it is ... the obvious place to put your student id
sid=""
######################
if sid == "" :
    print("HEY! didn't I tell you to put your sid in the obvious place")
 

In [ ]:
import random
n= 8 +1  ###highest number  reached by students counting off from zero (need +1)
ids=np.arange(n)
random.shuffle(ids)

n2=int(len(ids)/2)
print(ids[:n2])
print(ids[n2:])

In [ ]:
cquiz('mort2-partners')

## Carrying capacity, though a slippery concept is another approach to estimating pre-columbian populations.

Malthus does not use the term "carrying capacity" but when he talks says :

>"That population cannot increase without the means of subsistence
... That population does invariably increase where there are the means
of subsistence, ...  And that the superior power of population cannot be checked
without producing misery or vice..."

that is what he is talking about.  In chap 3, he links carrying capacity to state of nature -- which we would call "technology"  pastoralists v hunters v modern societies

Taking a Malthusian view one might  estimate pre contact population by estimating the maximum population that might be supported given the land and technology  available.  This sort  of estimate is generally over small areas where there is some specific knowledge of the habits of the indigenous (generally nonhuman) population.

For the Americas, applying the awesome power of gross generalities,  one can apply a thumb rule that the hunter-gather populations tend to live with roughly .1 to 1 KM^2 per person. At that level, the 48million KM^2 of the Americas would hold between 4.8 and 48 million people. 

Once agriculture begins,  those numbers would of course rise quickly which brings us to the chicken vs egg / Malthus vs Boserup question:

## Does technological change drive population...or does population drive technological change?




## Are the low or high counters implicitly assuming a Malthusian or Boserupian position ?




##  Malthus v Boserup and mortality crises



<div style="float:left;width:100%;">
<img src="http://courses.demog.berkeley.edu/mason88/images/blackdeath_main.jpg" style="width:50%;float:left;"/>
<img src="http://courses.demog.berkeley.edu/mason88/images/popdips2.gif" style="width:50%;float:left;clear:right"/>

</div>

<p>
For what it's worth,  
When Europe experienced the Black death of 1346-9 the population fell by 1/3 to 1/2 in just a few years, then
recovered in about 2 centuries (some places faster some slower). During those centuries the Black Death returned many times.  The last epidemic in England was in 1666. 
    
The reduced labor supply from the plague is credited by some historians as greatly increasing the bargainig position/welfare of peasants which brought about the end of serfdom in western europe. 
<p>
It is worth considering the experience of Europe during the Black Death of 1346-9 because at least some of the high counters contend that the populations of the new and old worlds were both of similar size AND at similar levels of technological development. One might thus expect that their experiences with massive mortality shocks might also be similar.
<p>
Of course there were significant differences as well.  European societies were not subsequently colonized by an exploitive invading power. But perhaps most of the western hemisphere natives weren't either.  The 10 - 12.5 million original North Americans were not afterall, actually seen by Europeans.  Dobyns et. al. hypothesize that these populations were quite distant from the coasts and were decimated by diseases *before* Europeans found them.


Consider also that according to Mann, european settlers and their decendents were also subject to epidemics.  Yet their populations appear to have been quite resiliant.  Immigration could explain some of this, but not with death rates in the 95% range.




There is farily wide agreement among demographers that in the aftermath of a mortality crisis there is generally some sort of fertility increase.  A recent example of this is in a paper by Jenna Nobles and colleagues https://www.ncbi.nlm.nih.gov/pmc/articles/PMC4411230/ which looks at the effect of the 2004 Tsunami in Indonesia. 

For Malthus --  a mortality crisis increases the resources available per living person and this induces the happier (poor) folks to marry younger reproduce more.


### Without a reason to the contrary,  why wouldn't we (as malthusians) expect the populations to recover at least a little bit in 400 years?


# Data science meets life tables


Last week we did a bunch of  17th Century algebra to compute life expectancies and probabilities of various type of family mortality experiences. Today we'll compute some more complicated and yet interesting numbers but this time using a more data sciency approach. In place of the tedious calculations,  we'll use the life table to generate simulated data from which we can learn what we are interested in.

### Proof of concept:  computing $e_x$  by simulation.

We'll first generate a simulated population which dies in accordance with the age pattern mortality described by a life table from the HMD.  From this simulated population whose dates of death we know, we can compute $e_x$ AND all sorts of interesting values which characterize what life (and death) was like over the centuries.

To simulate ages at death, we'll work with a quantity derivable from the $l_x$ : 

#### The unconditional  probability of death at any age x  
$pdeath_x = \frac{l_{x}-l_{x+1}}{l_0}$

This is very similar to $q_x$ -- which is the probability of death before age x+1 __conditional on having survived to age x__  our new quantity, $pdeath_x$ is the __unconditional__ probability of death between ages x and x+1. In Algebra $q_x$ has the same numerator but a different denominator, $ q_x=\frac{l_x - l_{x+1}}{l_x} $.

In the next cell we'll create a function that computes 'pdeath' for each age in a life table.  We'll append it as an additional column because it's easiest to keep track of that way.

Then we'll use that pdeath to create simulated birth cohorts.

In [ ]:
# Read some data from old Sweden
sweden=Table().read_table(datasite+'mortalitySWEc.csv')
#The sweden table contains many life tables one for each year from 1751 to 1917
swedenP=Table().read_table(datasite+'mortalitySWEp.csv')
# we only have period data for the US
usaP=Table().read_table(datasite+'mortalityUSp.csv')
# let's extract just the life table for the 1751 birth cohort
swe1751=sweden.where('Year',1751)

usa2015=usaP.where('Year',2015)
usa2015.show()

In [ ]:
ltab = usaP.where('Year',1998)
ltab.show(5)
def pdeathF(ltab):
    '''
    adds a column pdeath to life table ltab which gives the unconditional probability of death
    at age x
    
    pdeath=[]
    for age in ltab['Age'][0:-1] :
        
        ???????
        
    return(ltab.append_column('pdeath',pdeath))
    '''

In [ ]:
usa2015=pdeathF(usa2015)
usa2015

In [ ]:
## Using the pdeathF  function ...
usa2015=pdeathF(usa2015)
#swe1751.plot('Age', select="pdeathx")

plt.rcParams['figure.figsize']=(10,6)
plt.subplot(1,2,1)
plt.plot(usa2015['Age'],usa2015['qx'], label='qx')
plt.plot(usa2015['Age'],usa2015['pdeathx'],label='peathx')
plt.title("USA 2015")
plt.legend()

swe1751=pdeathF(swe1751)
plt.subplot(1,2,2)
#swe1751.plot('Age', select="pdeathx")
plt.rcParams['figure.figsize']=(6,6)
plt.plot(swe1751['Age'],swe1751['qx'], label='qx')
plt.plot(swe1751['Age'],swe1751['pdeathx'],label='peathx')
plt.title("Sweden 1751")
plt.legend()
plt.show()
plt.show()
##note that pdeath and qx are the same in row Age 0  but not so much thereafter

## What is the difference between qx and pdeath ?

### Now let's use pdeath to compute $e_0$ aka _Life expectancy at birth_  for the 1751 birth cohort


Recall that previously, we conceptualized $e_x$ as the sum of the average number of person years lived by cohort members at each year of age. That's perfectly fine -- it turns out, however, that that is a round about way of getting at what $e_x$ is really about -- which is the __average duration of life__.  In fact, if John Graunt had had a computer, he no doubt would have simply taken the average of the ages at death of all the people in his data set -- rather than bothering to invent the life table.  

So our task then is to use the life table to _simulate_ the data from which the life table was created. Then to use that simulated data to simply and elegently compute stuff that one might otherwise laboriously calculate using the lifetable.

#### Proof of concept:  simulate data on lifetimes; take mean; compare to $e_x$


In the cell below we use the .sample method to draw a sample (with replacement) of rows of the swe1751 life table (we don't need the entire row -- but that's how .sample() works. 

The clever part is that we'll use the new 'pdeath' column as weights.  This means that our resulting sample will have lots of copies of the rows in the life table -- but it will have more copies of rows for which 'pdeath' is higher.  If we let __each row in the sample represent a simulated person__, then we can take their 'Age' as their age at death -- or more precisely their age at their last birthday before death.  We can improve our estimate of the age at death by adding the $a_x$ column to the Age column. The $a_x$ column,  is still the average day of death measured as a proportion of a year. 

In [ ]:
## simulating ages at death

## here we are taking the weighted sample
pop1751=swe1751.sample(k=100000,with_replacement=True,weights=swe1751['pdeathx'])
## now we just take the mean
e0_good=np.mean(pop1751['Age'])
## add the ax to improve the estimate
e0_better=np.mean(pop1751['Age']+pop1751['ax'])
## extract the official ex from the original life tab;e
e0_hmd=swe1751.where('Age',0)['ex']
print("""
e0 ignoring years of life lived during the age interval of death:{0}
e0 using 'ax' which is the estimated number of years lived 
during the age interval by those who die in it:{1}
e0 from the human mortality database:{2}

Run this cell again ...""".format(e0_good,e0_better,e0_hmd))



### So we now have a simple DS8 compliant way of computing $e_0$ as the average simulated date of death.  But the $e_x$ column gives the expected duration of life _after any age x_.  How can that be done using our simulated data method? -- write code to estimate $e_20$



In [ ]:
## here we are taking the weighted sample
pop1751=swe1751.sample(k=100000,with_replacement=True,weights=swe1751['pdeathx'])
## now we just take the mean
pop20=pop1751.where(pop1751['Age'] >= 20)
e20_better=np.mean(pop20['Age']+pop20['ax'])
print("estimate of e_20=",e20_better)

print(swe1751.where('Age',20))
print("Hmmm what's wrong here?")

## Using life tables to explore family level mortality experience

In what remains of this lab, we'll use the technique we developed above to generate simulated ages at death of people born in various years to estimate the probabilities of things like:  A mother born in year X outliving her child born in year Y -- or the average number of years of life together that a family of 5 might expect to enjoy.

To do this we will generally assume that the probabilities of death among family members are *independtent* because it is a lot easier.  This is probably not entirely inocuous since there are  many reasons to expect that long lived parents are more likely to have long lived children:

1. Genetic proclivities toward health and illness exist.
1. Exposure to pathogens might correlate to residence.
1. Cultural habits might correlate with health.
1. Human capital of parents could affect longevity of both generations.

It takes courage to be a scientist. But it also takes sense.  We'll examine the consequences of the independence assumption before just taking it as given.


## Let's write a function that creates a simulated set of ages at death for a birth cohort

like what we have been doing for minutes now.

In [ ]:
# a function to generate a simulated population
def popSim(ltab,N=100000):
    """
    uses pdeathF to add a column of probabilities of death; then draws N ages (at death) 
    and returns ages at death as vector
    """
    ltab0=pdeathF(ltab)
    ## zero probability of death is fine for ages to which no one survives -- but
    ## .sample() cannot stand it so we remove them from consideration
    ltab0=ltab0.where('pdeathx',are.above(0))
 
    ltabS=ltab0.sample(k=N,with_replacement= True, weights= ltab0['pdeathx'])
    aged=ltabS['Age']+ltabS['ax']
    return(aged)

In [ ]:
# example 0 - expectation of remaining years of life at age 20 for 1789 Sweden (cohort life tables)
# extract the 1789 life table from sweden
swe1789=sweden.where('Year',1789)
## generate a simulated array of death ages
aged=popSim(swe1789)
aged=aged[aged >=20] -20
print(aged.mean())
swe1789.where('Age',20)


In [ ]:
## Example 1 - probability of person born in 1760 living to age 44 
swe1760c=sweden.where('Year',1760)
aged=popSim(swe1760c)
np.mean(aged >= 44) 
#note this trick aged is an array of ages at death; aged >= 44 is an array of True/False by 
# python convention True = 1 and False = 0 ; the mean of a bunch of 1s and 0s is the proportion
# of 1s which in this situation is an estimate of the probability that an element of aged >= 44

In [ ]:
# Example 1.1 - probability of a person born in 1760 AND who survives until age 1 
# also surviving until age 44
# select only the lifetimes greater than 1 year long
aged1plus=aged[aged >= 1]
# take the mean of the subset of lifetimes
np.mean(aged1plus >= 44)

In [ ]:
# Example 1.1a (equivalent to 1.1 but different algebra)
np.mean(aged >= 44)/np.mean(aged >= 1)

In [ ]:
usaP.where('Year',1933)

In [ ]:
# Example 2 probability that a pair of twins born in 1933 will celebrate their 
# 20th birthday together using period USA data 
usa1933=usaP.where('Year',1933)
aged=popSim(usa1933)

np.mean(aged >= 20 )**2

##  The independence assumption

So far, we have assumed that longevity is independently distributed according to the life table.  Yet we know that longevity has a inherited componant (note inherited does not solely mean genetic). The degree of heritability is debated but most estimates are in the 15 to 30 percent range.  We can think of heritability as predictability.  If knowhing the longevity of the mother allows us to predict the longevity of the child, that would strongly imply that some mechanism is at work whereby long lived parents tend to have long lived children.

This is exactly analagous to what you will see in Chapter 15 of the Data 8 text book where the height of parents and children are shown to be "correlated".  

To preview chapter 15.1 in a dry and boring way  consider this definition of the Pearson correlation coefficient

$$ \rho{}_{x,y} =\frac{ \Sigma{}[(\bar{x}-\mu{}_x)(\bar{y}-\mu{}_y)]}{\sigma{}_x\sigma{}_y}$$

where $\bar{x}$ is the mean of x and $\sigma{}x$ is the standard deviation of x

$\rho{}$ varies between -1 and 1  when correlation is 1 or -1 a linear function describes the relationship between x and y when $\rho{}$ is 0, no relationship is evident that is, knowing the value of x tells you nothing about the value of the corresponding y

### Consider the longevity of siblings

Like parents, siblings share on average half of their genes and a lot of their environment.


Let's simulate twins with and without correlation

In [ ]:
## twins with UNcorrleated longevity
sib1 = popSim(usaP.where('Year',1933))
sib2 = popSim(usaP.where('Year',1933))
firstdeath = np.minimum(sib1,sib2)
rho=np.corrcoef(sib1,sib2)
print('mean time to first death {} rho= {}'.format(np.mean(firstdeath),rho))
plt.hist(firstdeath)
plt.show()
plt.plot(sib1,sib2,'ob')
plt.show()


In [ ]:
# Example 3  probability that siblings born 3 years apart who celebrate 
# their 7th and 10th birthdays together will ALSO  Celebrate thie 27th and 30th 
# birthdays together. Using cohort data with older sib born in 1850 and younger sib in 
# 1853
aged1=popSim(sweden.where('Year',1850))
aged2=popSim(sweden.where('Year',1853))
(np.mean(aged1 >= 30)/np.mean(aged1 >= 10)) *(np.mean(aged2 >=27)/np.mean(aged2 >= 7))

In [ ]:
## Example 4  probability that a woman born in 1765 is outlived by child 
## born when she is 22 years old (in 1787 )

# for this it will be easier to work with year of death
yrdmom=popSim(sweden.where('Year',1765)) + 1765
# delete moms who die before child's birth year
yrdmom=yrdmom[yrdmom >= 1787]

# one kid for each mom who survives until birth year
yrdkid=popSim(sweden.where('Year',1787),N=len(yrdmom)) + 1787

# So now we have years of deaths of moms, yrdmom and years of death of kids, yrdkid
# We can let the first element in yrdmom refer to the mom of the first element of yrdkid
# and so on... then all we need to do is ask how many yrdkid elements exceed their corresponding
# yrdmom value ?

# because numpy is cool, we can simply write:
result=np.mean(yrdkid > yrdmom)
print("The probability that a mom born in 1765 will pre-decease her child born in 1787:{0}".format(result))

## For you edification only -- if you prefer to ignore this numpy trick and do it the long way 
## here's the code that gets you the same result as np.mean(yrdkid > yrdmom)

# we now have arrays of mom's and kid's years of death
# loop through and add up those for whom condition is met
result=[]
for i in np.arange(len(yrdkid)):
    famres= yrdkid[i]> yrdmom[i]
    result.append(famres)
    

result   
np.mean(result)



# this will work because numpy can do tricky things
#np.mean(yrdmom < yrdkid)



In [ ]:
## Example 4.1   probability that a woman born in 1765 is outlived by child 
## born when she is 22 years old (in 1787 )  BY MORE THAN 20 years

## same as example 4 BUT instead of merely outliving mom, this time junior has to live 20 
## more years than mom

# for this it will be easier to work with year of death
yrdmom=popSim(sweden.where('Year',1765)) + 1765
# delete moms who die before child's birth year
yrdmom=yrdmom[yrdmom >= 1787]

# one kid for each mom who survives until birth year
yrdkid=popSim(sweden.where('Year',1787),N=len(yrdmom)) + 1787

# So now we have years of deaths of moms, yrdmom and years of death of kids, yrdkid
# We can let the first element in yrdmom refer to the mom of the first element of yrdkid
# and so on... then all we need to do is ask how many yrdkid elements exceed their corresponding
# yrdmom value ?

# because numpy is cool, we can simply write:
#result=np.mean(yrdkid > yrdmom )
result=np.mean(yrdkid > (yrdmom + 20 ))

print("""The probability that a mom born in 1765 will pre-decease 
her child born in 1787 by at least 20 years :{}""".format(result))









## What is the median number of years that a woman will spend as a widow with no living children.

1. born in Sweden in 1800
1. husband 10 years older
1. child born 1825
1. child born 1830

## How about the 95 percentile

In [ ]:
# example 5

## What is the median number of years that a woman will spend as a 
## widow with no living children.
# ASSUMING:
# born in Sweden in 1800
# husband 10 years older
# child born 1825
# child born 1830

## How about the 95 percentile
ego=popSim(sweden.where('Year',1800),10000) + 1800
husband=popSim(sweden.where('Year',1790),10000) + 1790
child1=popSim(sweden.where('Year',1825),10000) + 1825
child2=popSim(sweden.where('Year',1830),10000) + 1830
RES=[]
for i in np.arange(len(ego)):
    oldestkin=max( husband[i],child1[i],child2[i])
    if ego[i] > oldestkin:
        res=ego[i]-oldestkin
    else:
        res=0
    RES.append(res)
print("median", np.percentile(RES,50))
print("95th percentile", np.percentile(RES,95))
    

## Problem 1 (of two)

### What is the probability that twins will outlive their mother if the following circumstances hold

1. That the mother is born in Sweden in 1800 and the twins are born in 1820.  
1. We are assuming that longevity is independent in all cases -- giving birth does not affect mother's survival and sharing a womb with an obnoxious sibling does not affect child's survival.  As we know, neither is true in 1800 Sweden--but it's OK, we are scientists.



FYI Using period data for the US, a woman born in 1995 will be outlived by both twins born in 2015 is about .75

A  start on this problem is provided below. Definitely use the np.random.seed(998877) call in order to be able to check your result.

In [ ]:
# Probability that both twins will outlive their mother 
# if she is born in 1800 and they in 1820.

## here is a start (remove tripple quotes obviously):

# very important to set random.seed
# set random seed so that your answer will be the same as mine
'''
np.random.seed(998877)
## sweden 1800
# generate year of death for simulated sample of mothers
yrdmom=popSim(sweden.where('Year',1800)) + 1800
yrdmom=yrdmom[yrdmom >= ???]
# generate year of death for simulated twins
yrdkid0=popSim(sweden.where('Year',1820),N=len(yrdmom)) + 1820
yrdkid1=popSim(sweden.where('Year',1820),N=len(yrdmom)) + 1820

famres = ( ? >  ?) & (? > ?)
np.mean(?)
'''

## Problem 2  -- duration of family life

What is the average number of years that family composed of two  parents and two children will live  before ONE member of the family dies? In other words what is the average number of years that these four people will be simultaneously alive.

Let's assume the following:

Everyone is a Swede 
mother born 1780
father born 1782
child0 born 1805
child1 born 1807

Let's further assume:

That both parents are alive at the birth of the second child, but let's NOT assume that the first child survives until the second child is born (this is easier to compute)

count as zero years families in which the first child does not survive until the birth of the second (not negative)

The corresponding value for contemporary US is around 43 years



In [ ]:
## A start on  problem 2
# set random seed so you and I get the same answer
np.random.seed(36251)
# years of death of mom and dad
yrdmom=popSim(sweden.where('Year',1780)) + 1780
yrddad=popSim(sweden.where('Year',1782)) + 1782
# both parents must live until birth of second child
# logical indicating both mom and dad are alive in 1807
alive07=(yrdmom >= 1807) & (yrddad >=1807)
## dropping dead parents
yrddad=yrddad[alive07]
yrdmom=yrdmom[alive07]
# years of death of two kids 
ydkid0=popSim(sweden.where('Year',1805),N=np.sum(alive07)) + 1805
ydkid1=popSim(sweden.where('Year',1807),N=np.sum(alive07)) + 1807

# array to hold result
result=[]
## remove the tripple quotes and write a few lines of code
"""
for i in np.arange(np.sum(alive07)) :
    CODE GOES HERE
"""

In [ ]:
cquiz('mort2-python1')

In [ ]:
cquiz('mort2-python2')

In [ ]:
cquiz('mort2-qxpdeath')

In [ ]:
cquiz('mort2-simwhy')

In [ ]:
cquiz('mort2-tmomindep')

In [ ]:
cquiz('mort2-tmomdep')

In [ ]:
cquiz('mort2-famlife')

In [ ]:
cquiz('mort2-famlife25')

In [ ]:
cquiz('mort2-eval')